In [ ]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(1, os.path.abspath(os.pardir))
import CONFIG

%reload_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import Image

import pandas as pd
import numpy as np
import cv2
import os, re

import torch

import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset

from matplotlib import pyplot as plt 
plt.rcParams['figure.figsize'] = (10.0, 10.0)

In [ ]:
DATA_DIR = CONFIG.CFG.DATA.BASE
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))
train_df.shape, test_df.shape

In [ ]:
NEW_COLUMNS = ['x', 'y', 'w', 'h']

# initialize new columns with -1
for new_column in NEW_COLUMNS:
    train_df[new_column] = -1

# expand the bbox coordinates into x, y, w, h
def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x), dtype=np.float)
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train_df[NEW_COLUMNS] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x)))
train_df.drop(columns=['bbox'], inplace=True)

In [ ]:
train_df.dtypes

In [ ]:
def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    assert image is not None, f"IMAGE NOT FOUND! AT {image_path}"
    return image

def load_image_from_folder(image_id, folder="train"):
    path = os.path.join(INPUT_DIR, folder, f"{image_id}.jpg")
    return load_image(path)

def read_image_from_path(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def read_image_from_train_folder(image_id):
    path = os.path.join(INPUT_DIR, "train", f"{image_id}.jpg")
    return read_image_from_path(path)

In [ ]:
sample_image_id = "b6ab77fd7"
plt.imshow(read_image_from_train_folder(sample_image_id))
_ = plt.title(sample_image_id)

In [ ]:
def draw_boxes_on_image(boxes, image, color=(255,0,0)):
    for box in boxes:
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, 3
        )
    return image

In [ ]:
sample_image_id = train_df.image_id.sample().item()
sample_image = read_image_from_train_folder(sample_image_id)
sample_bounding_boxes = train_df[train_df.image_id == sample_image_id][["x", "y", "x2", "y2"]]

plt.imshow(draw_boxes_on_image(sample_bounding_boxes.to_numpy(), sample_image, color=(0, 200, 200)))
plt.title(sample_image_id)

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_channels=in_features, num_classes=2)

model.roi_heads

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def move_batch_to_device(images, targets):
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    return images, targets


In [ ]:
unique_image_ids = train_df["image_id"].unique()

n_validation = int(0.2* len(unique_image_ids))
valid_ids = unique_image_ids[-n_validation:]
train_ids = unique_image_ids[:-n_validation]

validation_df = train_df[train_df['image_id'].isin(valid_ids)]
trainining_df = train_df[train_df['image_id'].isin(train_ids)]

print("%i training samples\n%i validation samples" % (len(trainining_df.image_id.unique()), len(validation_df.image_id.unique())))

In [ ]:
class WheatDataset(Dataset):
    def __init__(self, dataframe):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
        image = read_image_from_train_folder(image_id).astype(np.float32)
        image /= 255.0
        # change the shape from [h,w,c] to [c,h,w]  
        image = torch.from_numpy(image).permute(2,0,1)

        records = self.df[self.df['image_id'] == image_id]

        boxes = records[['x', 'y', 'x2', 'y2']].values
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        n_boxes = boxes.shape[0]

        labels = torch.ones((n_boxes,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels

        return image, target

In [ ]:
train_dataset = WheatDataset(trainining_df)
valid_dataset = WheatDataset(validation_df)

def collate_fn(batch):
    return tuple(zip(*batch))

is_training_on_cpu = device == torch.device('cpu')
batch_size = 4 if is_training_on_cpu else 16

train_data_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
batch_of_images, batch_of_targets = next(iter(train_data_loader))

sample_boxes = batch_of_targets[0]['boxes'].cpu().numpy().astype(np.int32)
# convert to normal image format
sample_image = batch_of_images[0].permute(1,2,0).cpu().numpy()

plt.imshow(draw_boxes_on_image(sample_boxes, sample_image, color=(0,200,200)))


# %%
optimizer = torch.optim.SGD(model.parameters(), lr=0.007, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


# %%
num_epochs = 1 if is_training_on_cpu else 5

model = model.to(device)
model.train()

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    average_loss = 0
    for batch_id, (images, targets) in enumerate(train_data_loader):
        images, targets = move_batch_to_device(images, targets)

        loss_dict = model(images,targets)
        batch_loss = sum(loss for loss in loss_dict.values()) / len(loss_dict)

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        lr_scheduler.step()

        loss_value = batch_loss.item()
        average_loss = average_loss + (loss_value - average_loss) / (batch_id + 1)

        print("Mini-batch: %i/%i Loss: %.4f" % ( batch_id + 1, len(train_data_loader), average_loss), end='\r')
        if batch_id % 100 == 0:
            print("Mini-batch: %i/%i Loss: %.4f" % ( batch_id + 1, len(train_data_loader), average_loss))

In [ ]:
model.eval()

def make_validation_iter():
    valid_data_iter = iter(valid_data_loader)
    for images, targets in valid_data_iter:
        images, targets = move_batch_to_device(images, targets)

        cpu_device = torch.device("cpu")
        outputs = model(images)
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        for image, output, target in zip(images, outputs, targets):
            predicted_boxes = output['boxes'].cpu().detach().numpy().astype(np.int32)
            ground_truth_boxes = target['boxes'].cpu().numpy().astype(np.int32)
            image = image.permute(1,2,0).cpu().numpy()
            yield image, ground_truth_boxes, predicted_boxes

validation_iter = make_validation_iter()


In [ ]:
image, ground_truth_boxes, predicted_boxes = next(validation_iter)
image = draw_boxes_on_image(predicted_boxes, image, (255,0,0))
image = draw_boxes_on_image(ground_truth_boxes, image , (0,255,0))
plt.imshow(image)

In [ ]:
torch.save(model.state_dict(), os.path.join(CONFIG.CFG.DATA.OUT_MODELS, 'simple_fastercnn.pth'))

In [ ]:
CONFIG.upload_to_kaggle("wheatfastercnn", "Simple Faster R-CNN")